In [416]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/msbd5001-fall2019/samplesubmission.csv
/kaggle/input/msbd5001-fall2019/test.csv
/kaggle/input/msbd5001-fall2019/train.csv


In [417]:
trainFile = '/kaggle/input/msbd5001-fall2019/train.csv'
testFile = '/kaggle/input/msbd5001-fall2019/test.csv'
X_trainFull = pd.read_csv(trainFile)
X_testFull = pd.read_csv(testFile)

In [418]:
X_trainCopy = X_trainFull.copy()
X_testCopy = X_testFull.copy()

In [419]:
def convert_purchase(df):
    addCol = df['purchase_date'].str.split(" |, ")
    df['purchase_month']= addCol.str[0].map({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})


In [420]:
convert_purchase(X_trainCopy)
convert_purchase(X_testCopy)

In [421]:
cols_with_missing = [col for col in X_trainCopy.columns if X_trainCopy[col].isnull().any()] 
print(cols_with_missing)

missing_val_count_by_column = (X_trainCopy.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

cols_with_missing = [col for col in X_testCopy.columns if X_testCopy[col].isnull().any()] 
print(cols_with_missing)

missing_val_count_by_column = (X_testCopy.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

['purchase_date', 'total_positive_reviews', 'total_negative_reviews', 'purchase_month']
purchase_date             2
total_positive_reviews    2
total_negative_reviews    2
purchase_month            2
dtype: int64
['purchase_date', 'total_positive_reviews', 'total_negative_reviews', 'purchase_month']
purchase_date             2
total_positive_reviews    2
total_negative_reviews    2
purchase_month            2
dtype: int64


In [422]:
#drop rows contains
X_trainCopy2 = X_trainCopy.dropna(axis=0,how='any')

fill_4 = X_testCopy.purchase_month.mode().iloc[0]
fill_5 = X_testCopy.total_positive_reviews.mean()
fill_6 = X_testCopy.total_negative_reviews.mean()

X_testCopy2 = X_testCopy.fillna({'purchase_month': fill_4,'total_positive_reviews':fill_5,'total_negative_reviews':fill_6})

In [423]:
X_trainCopy2['zscore_of_price'] = (X_trainCopy2.price - X_trainCopy2.price.mean())/X_trainCopy2.price.std(ddof=0)

X_trainCopy3 = X_trainCopy2[abs(X_trainCopy2['zscore_of_price']) < 3]

X_trainCopy3.drop(['zscore_of_price'], axis=1, inplace = True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [424]:
X_trainTemp1 = X_trainCopy3['genres'].str.get_dummies(sep=',')
X_testTemp1 = X_testCopy2['genres'].str.get_dummies(sep=',')
moreCol1_1 = [col for col in X_trainTemp1.columns] 
moreCol1_2 = [col for col in X_testTemp1.columns]
difference1_train = [item for item in moreCol1_1 if item not in moreCol1_2]
difference1_test = [item for item in moreCol1_2 if item not in moreCol1_1]

X_trainTemp2 = X_trainCopy3['categories'].str.get_dummies(sep=',')
X_testTemp2 = X_testCopy2['categories'].str.get_dummies(sep=',')
moreCol2_1 = [col for col in X_trainTemp2.columns] 
moreCol2_2 = [col for col in X_testTemp2.columns] 
difference2_train = [item for item in moreCol2_1 if item not in moreCol2_2]
difference2_test = [item for item in moreCol2_2 if item not in moreCol2_1]

In [425]:
X_trainCopy4 = pd.concat([X_trainCopy3, X_trainTemp1,X_trainTemp2], axis=1, sort=False)
X_testCopy3 = pd.concat([X_testCopy2, X_testTemp1,X_testTemp2], axis=1, sort=False)

# X_trainCopy4 = pd.concat([X_trainCopy3, X_trainTemp1], axis=1, sort=False)
# X_testCopy3 = pd.concat([X_testCopy2, X_testTemp1], axis=1, sort=False)

for i in difference1_train:
    X_testCopy3[i]=0
for j in difference2_train:
    X_testCopy3[j]=0


In [426]:
#drop columns
dropSet = ['is_free','tags','purchase_date','release_date','genres','categories']

X_trainCopy5 = X_trainCopy4.drop(dropSet,axis=1)
X_testCopy5 = X_testCopy3.drop(dropSet,axis=1)

In [427]:
# split data for validation
y = X_trainCopy5.playtime_forever
X_trainCopy6 = X_trainCopy5.drop(['playtime_forever'], axis = 1)

In [428]:
 X_train, X_valid, y_train, y_valid = train_test_split(X_trainCopy6, y,
#                                                   train_size=0.9, test_size=0.1,
                                                      random_state=0)

In [429]:
#  for i in range(60):
    
#     s = (i+1)*5
model = RandomForestRegressor(n_estimators = 180, random_state=0)

# model.fit(X_train, y_train)
# preds = model.predict(X_valid)

model.fit(X_trainCopy6, y)


# print(s,np.sqrt(mean_squared_error(y_valid, preds)))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=180,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [430]:
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(model,X_trainCopy6,y,cv = 5,scoring = 'neg_mean_squared_error')

# rmse = np.sqrt(-scores)
# print(np.sqrt(-scores))

# print(rmse.mean())

In [431]:
predsTest = model.predict(X_testCopy5)
output = pd.DataFrame({'id': X_testCopy5.index,
                   'playtime_forever': predsTest})

output.to_csv('submission.csv', index=False)